# 🔬 ANÁLISE TEMPORAL DE MUNICIPIOS - PROJETO CAIPORA

<div align="center">
    <img src="image.png">
</div>

## 0. Pré requisitos para funcionamento das respectivas partes

### Importações de bibliotecas

In [26]:
from ipywidgets import Dropdown, VBox, Button, Output, Text
from IPython.display import display
from pysus.online_data import IBGE
import pandas as pd

from merradownload.merra_scraping import baixar_merra

## IBGE

In [37]:
print(IBGE.list_agregados().tail(40))

Fetching Data groupings from https://servicodados.ibge.gov.br/api/v3/agregados?
    id                                               nome  \
29  AC          Pesquisa Anual da Indústria da Construção   
30  PB                         Pesquisa Anual de Comércio   
31  PY                         Pesquisa Anual de Serviços   
32  PP                     Pesquisa da Pecuária Municipal   
33  AM           Pesquisa de Assistência Médico-Sanitária   
34  ES                               Pesquisa de Estoques   
35  PM         Pesquisa de Informações Básicas Municipais   
36  IT                               Pesquisa de Inovação   
37  OF                  Pesquisa de Orçamentos Familiares   
38  SH                 Pesquisa de Serviços de Hospedagem   
39  SE     Pesquisa de Serviços de Publicidade e Promoção   
40  PT   Pesquisa de Serviços de Tecnologia da Informação   
41  RC            Pesquisa Estatísticas do Registro Civil   
42  PI                          Pesquisa Industrial Anual   
43  P

## Dados da nasa


In [24]:
codigos_ibge_df = pd.read_csv(r"../data/codigos_ibge.csv", sep=";")

estados = sorted(codigos_ibge_df["uf"].unique())
municipios_por_estado = {
    uf: sorted(codigos_ibge_df[codigos_ibge_df["uf"] == uf]["municipio"].unique())
    for uf in estados
}

estados_dd = Dropdown(
    options=estados,
    description='Estados:',
    disabled=False
)

municipios_dd = Dropdown(
    options=municipios_por_estado[estados[0]],
    description='Municipios:',
    disabled=False
)

def atualiza_municipios(change):
    uf_selecionado = change['new']
    municipios_dd.options = municipios_por_estado[uf_selecionado]

estados_dd.observe(atualiza_municipios, names='value')

In [25]:
# Widgets para login e senha
login_txt = Text(description='Login:', placeholder='Usuário NASA')
senha_txt = Text(description='Senha:', placeholder='Senha NASA', style={'description_width': 'initial'}, layout={'width': '300px'},  password=True)

baixar_btn = Button(description="Baixar dados da NASA")
output = Output()

def baixar_dados_callback(b):
    municipio = municipios_dd.value
    uf = estados_dd.value
    dados_municipio = codigos_ibge_df[(codigos_ibge_df["uf"] == uf) & (codigos_ibge_df["municipio"] == municipio)]
    if not dados_municipio.empty:
        lat = float(dados_municipio["latitude"].values[0])
        lon = float(dados_municipio["longitude"].values[0])
        nome = municipio
        usuario = login_txt.value
        senha = senha_txt.value
        output.clear_output()
        with output:
            print(f"Baixando dados para {nome} ({lat}, {lon})...")
            baixar_merra(
                username=usuario,
                password=senha,
                years=[2020, 2021],
                field_id="T2M",
                field_name="temperatura",
                database_name="M2I1NXASM",
                database_id="inst1_2d_asm_Nx",
                locs=[(nome, lat, lon)],
                conversion_function=lambda x: x - 273.15,
                aggregator="mean"
            )
            print("Download finalizado!")
    else:
        output.clear_output()
        with output:
            print("Município não encontrado ou sem coordenadas.")

baixar_btn.on_click(baixar_dados_callback)

display(VBox([login_txt, senha_txt, estados_dd, municipios_dd, baixar_btn, output]))